## To do from last time

1. Check why all the Beta's are the same
   Is this true for them in the example code as well?
2. Double check the blog to see that we get what they get by using our code
3. Write out the analytical solution 
4. Estimate the analytical solution with a neural network
5. (Optional) Repeat for Lorenz 96 data if it works

*. (Added a week later) See how the "Advanced econometric metthods" book does it. 

## 1. Check why all the Beta's are the same. Is this true for them in the example code as well? 

Let us start by looking at the output from the GC paper (their repo)

![alt text](./gc.png)

Seems like they do. To further solidify our belief, we jump dive into their function "simulate_var" and print beta before and after the call to make_var_stationary:

In [16]:
import numpy as np
import pandas as pd
from IPython.display import display

def make_var_stationary(beta, radius=0.97):
    '''Rescale coefficients of VAR model to make stable.'''
    p = beta.shape[0]
    lag = beta.shape[1] // p
    bottom = np.hstack((np.eye(p * (lag - 1)), np.zeros((p * (lag - 1), p))))
    beta_tilde = np.vstack((beta, bottom))
    eigvals = np.linalg.eigvals(beta_tilde)
    max_eig = max(np.abs(eigvals))
    nonstationary = max_eig > radius
    if nonstationary:
        return make_var_stationary(0.95 * beta, radius)
    else:
        return beta

def simulate_var(p, T, lag, sparsity=0.5, beta_value=1.0, sd=0.1, seed=0):
    if seed is not None:
        np.random.seed(seed)

    # Set up coefficients and Granger causality ground truth.
    GC = np.eye(p, dtype=int)
    beta = np.eye(p) * beta_value

    num_nonzero = int(p * sparsity) - 1
    for i in range(p):
        choice = np.random.choice(p - 1, size=num_nonzero, replace=False)
        choice[choice >= i] += 1
        beta[i, choice] = beta_value
        GC[i, choice] = 1

    beta = np.hstack([beta for _ in range(lag)])
    print("Initial Beta:")
    beta_df = pd.DataFrame(beta)
    display(beta_df)
    beta = make_var_stationary(beta)
    print("Stationary Beta:")
    beta_df = pd.DataFrame(beta)
    display(beta_df)
    # Generate data.
    burn_in = 100
    errors = np.random.normal(scale=sd, size=(p, T + burn_in))
    X = np.zeros((p, T + burn_in))
    X[:, :lag] = errors[:, :lag]
    for t in range(lag, T + burn_in):
        X[:, t] = np.dot(beta, X[:, (t-lag):t].flatten(order='F'))
        X[:, t] += errors[:, t-1]

    return X.T[burn_in:], beta, GC

# Run the simulation
X_np, beta, GC = simulate_var(p=5, T=1000, lag=3)
print("With the corresponding Granger Caqusality:")
display(GC)


Initial Beta:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


Stationary Beta:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.14989,0.00000,0.00000,0.14989,0.00000,0.14989,0.00000,0.00000,0.14989,0.00000,0.14989,0.00000,0.00000,0.14989,0.00000
1,0.14989,0.14989,0.00000,0.00000,0.00000,0.14989,0.14989,0.00000,0.00000,0.00000,0.14989,0.14989,0.00000,0.00000,0.00000
2,0.00000,0.00000,0.14989,0.00000,0.14989,0.00000,0.00000,0.14989,0.00000,0.14989,0.00000,0.00000,0.14989,0.00000,0.14989
3,0.00000,0.14989,0.00000,0.14989,0.00000,0.00000,0.14989,0.00000,0.14989,0.00000,0.00000,0.14989,0.00000,0.14989,0.00000
4,0.00000,0.14989,0.00000,0.00000,0.14989,0.00000,0.14989,0.00000,0.00000,0.14989,0.00000,0.14989,0.00000,0.00000,0.14989


With the corresponding Granger Caqusality:


array([[1, 0, 0, 1, 0],
       [1, 1, 0, 0, 0],
       [0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1]])

Breaking down this part of the code:





In [ ]:
num_nonzero = int(p * sparsity) - 1
    for i in range(p):
        choice = np.random.choice(p - 1, size=num_nonzero, replace=False)
        choice[choice >= i] += 1
        beta[i, choice] = beta_value
        GC[i, choice] = 1

    beta = np.hstack([beta for _ in range(lag)])

shows us that beta is horizontally stacked range(lag) times. To visualize this we run

In [27]:
a = np.array([[1, 2, 3],[4,5,6],[7,8,9]])
b = np.hstack([a] * 3)
print(b)



[[1 2 3 1 2 3 1 2 3]
 [4 5 6 4 5 6 4 5 6]
 [7 8 9 7 8 9 7 8 9]]


And we conclude that they indeed have decided to go with the same granger causality for each lag. 

Q1: Are we interested in investigating this further? I.e, change up the code so that the different lags (can) have different values and investigate if we are able to say something more accurate about the strength of the GC? This can be achieved by updating the simulate_var function in the following way:

In [31]:
import numpy as np
from IPython.display import display
#Make var stays the same
def make_var_stationary(beta, radius=0.97):
    '''Rescale coefficients of VAR model to make stable.'''
    p = beta.shape[0]
    lag = beta.shape[1] // p
    bottom = np.hstack((np.eye(p * (lag - 1)), np.zeros((p * (lag - 1), p))))
    beta_tilde = np.vstack((beta, bottom))
    eigvals = np.linalg.eigvals(beta_tilde)
    max_eig = max(np.abs(eigvals))
    nonstationary = max_eig > radius
    if nonstationary:
        return make_var_stationary(0.95 * beta, radius)
    else:
        return beta

def simulate_var(p, T, lag, sparsity=0.2, beta_range=(-0.3, 0.3), sd=0.1, seed=0, zeroing_prob=0.5):
    if seed is not None:
        np.random.seed(seed)

    # Set up Granger causality ground truth.
    GC = np.eye(p, dtype=int)

    # Generate the beta matrix for the VAR process (with lags)
    beta = np.zeros((p, p * lag))

    for i in range(p):
        # Ensure self-dependency for all lags
        for j in range(lag):
            beta[i, i + j * p] = np.random.uniform(beta_range[0], beta_range[1])  # Self-interaction
        
        # Select other random variables that influence variable i
        num_nonzero = int(p * sparsity)  # This determines how many other variables influence i
        if num_nonzero > 0:
            choice = np.random.choice([x for x in range(p) if x != i], size=num_nonzero, replace=False)
            for j in range(lag):
                # Randomly decide whether to zero out the coefficient
                if np.random.rand() > zeroing_prob:  # Keep with probability (1 - zeroing_prob)
                    beta[i, choice + j * p] = np.random.uniform(beta_range[0], beta_range[1], size=num_nonzero)
                    GC[i, choice] = 1  # Update Granger causality matrix

    print("Initial Beta:")
    beta_df = pd.DataFrame(beta)
    display(beta_df)
    beta = make_var_stationary(beta)
    print("Stationary Beta:")
    beta_df = pd.DataFrame(beta)
    display(beta_df)

    # Generate data
    burn_in = 100
    errors = np.random.normal(scale=sd, size=(p, T + burn_in))
    X = np.zeros((p, T + burn_in))
    X[:, :lag] = errors[:, :lag]
    for t in range(lag, T + burn_in):
        X[:, t] = np.dot(beta, X[:, (t-lag):t].flatten(order='F'))
        X[:, t] += errors[:, t]

    return X.T[burn_in:], beta, GC

# Simulate a VAR(3) process with 3 variables
p = 5  # Number of variables
T = 100  # Length of the time series
lag = 3  # VAR(3)
sparsity = 0.5  # 50% sparsity 
beta_range = (-0.3, 0.3)  # Random coefficients between -0.3 and 0.3
zeroing_prob = 0.5  # Probability of zeroing out a coefficient (excluding self-interaction)

# Simulate data
X_np, beta, GC = simulate_var(p, T, lag, sparsity, beta_range, zeroing_prob=zeroing_prob)

print("With the corresponding Granger Caqusality:")
display(GC)

Initial Beta:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.029288,-0.037448,0.000000,0.235064,0.000000,0.129114,-0.069935,0.000000,0.175035,0.000000,0.061658,0.040827,0.000000,0.255358,0.000000
1,0.000000,-0.257378,0.000000,0.000000,0.000000,0.180546,-0.247722,0.000000,-0.015835,0.000000,0.132380,-0.287869,0.000000,0.107328,0.000000
2,0.000000,0.000000,0.049212,0.000000,0.000000,0.000000,0.041060,0.022424,0.000000,-0.026310,0.000000,0.000000,0.155169,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.070581,0.000000,0.000000,-0.240432,0.241409,0.067257,0.000000,0.000000,-0.197454,0.091884,0.070160,0.000000
4,0.000000,0.000000,0.000000,0.000000,-0.085109,0.000000,0.000000,-0.238773,-0.174674,0.150412,0.000000,0.000000,0.000000,0.000000,0.064698


Stationary Beta:


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.029288,-0.037448,0.000000,0.235064,0.000000,0.129114,-0.069935,0.000000,0.175035,0.000000,0.061658,0.040827,0.000000,0.255358,0.000000
1,0.000000,-0.257378,0.000000,0.000000,0.000000,0.180546,-0.247722,0.000000,-0.015835,0.000000,0.132380,-0.287869,0.000000,0.107328,0.000000
2,0.000000,0.000000,0.049212,0.000000,0.000000,0.000000,0.041060,0.022424,0.000000,-0.026310,0.000000,0.000000,0.155169,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.070581,0.000000,0.000000,-0.240432,0.241409,0.067257,0.000000,0.000000,-0.197454,0.091884,0.070160,0.000000
4,0.000000,0.000000,0.000000,0.000000,-0.085109,0.000000,0.000000,-0.238773,-0.174674,0.150412,0.000000,0.000000,0.000000,0.000000,0.064698


With the corresponding Granger Caqusality:


array([[1, 1, 0, 1, 0],
       [1, 1, 0, 1, 0],
       [0, 1, 1, 0, 1],
       [0, 1, 1, 1, 0],
       [0, 0, 1, 1, 1]])

Note: this is easily done by changing a few lines of the code from their repo and training the network. 